In [1]:
!pip install opencv-python mediapipe --quiet

In [2]:
import cv2
import mediapipe as mp
import math

In [3]:
finger_tips = [4, 8, 12, 16, 20]
finger_names = ["Thumb", "Index", "Middle", "Ring", "Pinky"]
gesture_map = {
    (0, 1, 1, 0, 0): "✌️ PEACE",
    (1, 0, 0, 0, 1): "🤙 HANG LOOSE / CALL ME",
    (1, 1, 0, 0, 0): "👆 LOSER",
    (1, 1, 1, 1, 1): "🖐️ HIGH FIVE",
    (0, 1, 1, 1, 1): "✋ TALK TO THE HAND",
    (0, 1, 0, 0, 0): "👉 YOU",
    (1, 0, 0, 0, 0): "👍 GOOD JOB / DISLIKE",
    (0, 0, 0, 1, 1): "✊ POWER TO...",
    (0, 0, 0, 0, 1): "🤞 GOOD LUCK",
    (0, 1, 0, 0, 1): "🤘 ROCK",
}

In [4]:
def get_direction(start, end):
    dx = end.x - start.x
    dy = end.y - start.y
    angle = math.degrees(math.atan2(-dy, dx))

    if -45 <= angle <= 45:
        return "Right"
    elif 45 < angle <= 135:
        return "Up"
    elif angle > 135 or angle < -135:
        return "Left"
    elif -135 <= angle < -45:
        return "Down"
    else:
        return "Unknown"

def process_hand(hand_landmarks, label):
    fingers = []
    if label == "Right":
        fingers.append(int(hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x))
    else:
        fingers.append(int(hand_landmarks.landmark[4].x > hand_landmarks.landmark[3].x))

    for tip in finger_tips[1:]:
        fingers.append(int(hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y))
    directions = []
    for tip in finger_tips:
        base = hand_landmarks.landmark[tip - 2]
        end = hand_landmarks.landmark[tip]
        directions.append(get_direction(base, end))

    return fingers, directions

In [5]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1)
cap = cv2.VideoCapture(0) 
prev_gesture = None

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)
    if result.multi_hand_landmarks and result.multi_handedness:
        hand_landmarks = result.multi_hand_landmarks[0]
        hand_label = result.multi_handedness[0].classification[0].label
        fingers, directions = process_hand(hand_landmarks, hand_label)
        gesture = gesture_map.get(tuple(fingers), "Unknown")
        
        if gesture != "Unknown" and gesture != prev_gesture:
            open_fingers = [finger_names[i] for i, val in enumerate(fingers) if val == 1]
            print("\n--- Hand Gesture Changed ---")
            print(f"Hand Side     : {hand_label}")
            print(f"Open Fingers  : {open_fingers}")
            print(f"Directions    : {dict(zip(finger_names, directions))}")
            print(f"Hand Sign     : {gesture}")
            prev_gesture = gesture

        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        if gesture != "Unknown":
            cv2.putText(frame, f"{gesture}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.imshow("Hand Sign Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()


--- Hand Gesture Changed ---
Hand Side     : Right
Open Fingers  : ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']
Directions    : {'Thumb': 'Up', 'Index': 'Up', 'Middle': 'Up', 'Ring': 'Up', 'Pinky': 'Up'}
Hand Sign     : 🖐️ HIGH FIVE

--- Hand Gesture Changed ---
Hand Side     : Right
Open Fingers  : ['Index', 'Middle', 'Ring', 'Pinky']
Directions    : {'Thumb': 'Up', 'Index': 'Up', 'Middle': 'Up', 'Ring': 'Up', 'Pinky': 'Up'}
Hand Sign     : ✋ TALK TO THE HAND

--- Hand Gesture Changed ---
Hand Side     : Right
Open Fingers  : ['Index', 'Pinky']
Directions    : {'Thumb': 'Up', 'Index': 'Up', 'Middle': 'Down', 'Ring': 'Down', 'Pinky': 'Up'}
Hand Sign     : 🤘 ROCK

--- Hand Gesture Changed ---
Hand Side     : Right
Open Fingers  : ['Index']
Directions    : {'Thumb': 'Up', 'Index': 'Up', 'Middle': 'Down', 'Ring': 'Down', 'Pinky': 'Down'}
Hand Sign     : 👉 YOU

--- Hand Gesture Changed ---
Hand Side     : Right
Open Fingers  : ['Index', 'Middle']
Directions    : {'Thumb': 'Right', 'Index